In [1]:
#%pip install -r requirements.txt

In [2]:
import numpy as np 
import pandas as pd
import polars as pl
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import lightgbm as lgb
import pickle as pkl
import  gc
import glob
from tqdm import tqdm 
from dateutil.relativedelta import relativedelta
import os

In [ ]:
#Helper functions
def reduce_mem_usage(df, int_cast=True, obj_to_category=False, subset=None):
    """
    Iterate through all the columns of a dataframe and modify the data type to reduce memory usage.
    :param df: dataframe to reduce (pd.DataFrame)
    :param int_cast: indicate if columns should be tried to be casted to int (bool)
    :param obj_to_category: convert non-datetime related objects to category dtype (bool)
    :param subset: subset of columns to analyse (list)
    :return: dataset with the column dtypes adjusted (pd.DataFrame)
    """
    start_mem = df.memory_usage().sum() / 1024 ** 2;
    gc.collect()
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
#     cols_none = subset if subset is  None else df.columns.tolist()
#     for col_non in tqdm(cols_none):
#         df[col_non] = df[col_non].fillna(-888)
    
    cols = subset if subset is not None else df.columns.tolist()

    for col in tqdm(cols):
        col_type = df[col].dtype

        if col_type != object and col_type.name != 'category' and 'datetime' not in col_type.name:
#             df[col] = df[col].fillna(-888)
            c_min = df[col].min()
            c_max = df[col].max()

#             # test if column can be converted to an integer
#             treat_as_int = str(col_type)[:3] == 'int'
#             if int_cast and not treat_as_int:
#                 treat_as_int = check_if_integer(df[col])
                
            treat_as_int = True
            if treat_as_int:
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8, errors='ignore')
                elif c_min > np.iinfo(np.uint8).min and c_max < np.iinfo(np.uint8).max:
                    df[col] = df[col].astype(np.uint8, errors='ignore')
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16, errors='ignore')
                elif c_min > np.iinfo(np.uint16).min and c_max < np.iinfo(np.uint16).max:
                    df[col] = df[col].astype(np.uint16, errors='ignore')
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32, errors='ignore')
                elif c_min > np.iinfo(np.uint32).min and c_max < np.iinfo(np.uint32).max:
                    df[col] = df[col].astype(np.uint32, errors='ignore')
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64, errors='ignore')
                elif c_min > np.iinfo(np.uint64).min and c_max < np.iinfo(np.uint64).max:
                    df[col] = df[col].astype(np.uint64, errors='ignore')
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16, errors='ignore')
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32, errors='ignore')
                else:
                    df[col] = df[col].astype(np.float64, errors='ignore')
        elif 'datetime' not in col_type.name and obj_to_category:
            df[col] = df[col].fillna('Mis')
            df[col] = df[col].astype('category')
    gc.collect()
    end_mem = df.memory_usage().sum() / 1024 ** 2
    print('Memory usage after optimization is: {:.3f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return df

def date_column_depth_0(df):
    date_columns = ['date_decision'] + [x for x in df.columns if x[-1] == 'D'] 
    df[date_columns] = df[date_columns].apply(pd.to_datetime, errors='coerce')
    df_diff = df[date_columns].apply(lambda col: (df['date_decision'] - col).dt.days)
    df_diff.columns = [f'Diff_{col}' for col in df_diff.columns]
    df = pd.concat([df, df_diff], axis=1)
    return df


def union_parquest(list_parq):
    df_list = [reduce_mem_usage(pd.read_parquet(i)) for i in list_parq]
    union_df = pd.concat(df_list)
    union_df = reduce_mem_usage(union_df)
    return union_df   

def gini(x):
    total = 0
    for i, xi in enumerate(x[:-1], 1):
        total += np.sum(np.abs(xi - x[i:]))
    return total / (len(x)**2 * np.mean(x))

In [3]:
class CFG:
    home_directory = os.path.expanduser('~/kaggle_HomeCredit/parquet_files/')
    train_data_path = os.path.join(home_directory, 'train/')
    test_data_path = os.path.join(home_directory, 'test/')
    
    train_applprev_path = 'train_applprev_*.parquet'
    train_base_path =  'train_base_*.parquet'
    train_credit_path = 'train_credit_bureau_*.parquet'
    train_debitcard_path = 'train_debitcard_1.parquet'
    train_deposit_path = 'train_deposit_1.parquet'
    train_other_path = 'train_other_1.parquet'
    train_person_path = 'train_person_*.parquet'
    train_static_path = 'train_static_*.parquet'
    train_tax__path = 'train_tax_registry_*.parquet'


In [4]:
def investigate_data(file_path):
    for file in file_path:
        file_name = os.path.basename(file)
        df = pl.read_parquet(file)
        print(f"ファイル名: {file_name}")
        print(df)

In [5]:
file_path = glob.glob(os.path.join(CFG.train_data_path,CFG.train_applprev_path))
investigate_data(file_path)

ファイル名: train_applprev_2.parquet
shape: (14_075_487, 6)
┌─────────┬─────────────────────┬──────────────────┬─────────────────────┬────────────┬────────────┐
│ case_id ┆ cacccardblochreas_1 ┆ conts_type_509L  ┆ credacc_cards_statu ┆ num_group1 ┆ num_group2 │
│ ---     ┆ 47M                 ┆ ---              ┆ s_52L               ┆ ---        ┆ ---        │
│ i64     ┆ ---                 ┆ str              ┆ ---                 ┆ i64        ┆ i64        │
│         ┆ str                 ┆                  ┆ str                 ┆            ┆            │
╞═════════╪═════════════════════╪══════════════════╪═════════════════════╪════════════╪════════════╡
│ 2       ┆ null                ┆ EMPLOYMENT_PHONE ┆ null                ┆ 1          ┆ 1          │
│ 2       ┆ null                ┆ EMPLOYMENT_PHONE ┆ null                ┆ 0          ┆ 1          │
│ 2       ┆ null                ┆ PRIMARY_MOBILE   ┆ null                ┆ 0          ┆ 0          │
│ 2       ┆ null                ┆ PR